<a href="https://colab.research.google.com/github/mokarramis/Apriori/blob/main/Copy_of_CRNN_model_for_persian_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! unzip /content/drive/MyDrive/Persian_digits/archive.zip -d /content/drive/MyDrive/Persian_digits/archive

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/83164.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/831695.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/8317019.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/83171.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/831758.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/8317787.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/831804.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/8318165.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/831853.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/8318687.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/831906.jpg  
  inflating: /content/drive/MyDrive/Persian_digits/archive/dataset/

In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint

In [ ]:
# Load Dataset
filenames = []
for dirname, _, files in os.walk('/content/drive/MyDrive/Persian_digits/archive/dataset'):
    for f in files:
        filenames = np.append(filenames, f)
        
num_samples = len(filenames)
print('number of samples: ', num_samples)

number of samples:  30000


In [ ]:
print(filenames)

['000001.jpg' '0000232.jpg' '000115.jpg' ... '9998582.jpg' '999885.jpg'
 '999936.jpg']


In [ ]:
char_list = '0, 1, 2, 3, 4, 5, 6, 7, 8, 9'

In [ ]:
def encode_to_labels(txt):
    # encoding each label into list of digits
    encoded_list = []
    for char in txt:
        encoded_list.append(char_list.index(char))
    
    return encoded_list

In [ ]:
#Preprocessing Data

path = '/content/drive/MyDrive/Persian_digits/archive/dataset/' 

# lists for training dataset
training_img = []                    # the images for training the model
training_txt = []                    # the labels
train_input_length = []              # the input of LSTM part of the model
train_label_length = []              # the label's length (4 to 7)
train_orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0                    # max length for our labels (in this case 7)

# we append 75 to train_input_length & valid_input_length bc the lstm input size is 75

In [ ]:
for file in filenames[:100000]:
    raw = Image.open(path + file)
    gray = ImageOps.grayscale(raw)
    img = np.array(gray)
    img = np.expand_dims(img , axis = 2)
    img = img/255.

    txt = file.split('.')[0]

    if len(txt) > max_label_len:
        max_label_len = len(txt)
    
    # split the dataset (85% train, 15% test)
    if np.random.rand() >= 0.85:
        valid_orig_txt.append(txt)
        valid_label_length.append(len(txt))
        valid_input_length.append(75)
        valid_img.append(img)
        valid_txt.append(encode_to_labels(txt))
    else:
        train_orig_txt.append(txt)
        train_label_length.append(len(txt))
        train_input_length.append(75)
        training_img.append(img)
        training_txt.append(encode_to_labels(txt))

In [ ]:
print(txt)

03418


In [ ]:
print(train_orig_txt)

['000001', '0000232', '000115', '000124', '00017', '00021', '000252', '0003094', '000316', '0003449', '00039', '000438', '00051', '0005397', '00055', '000616', '00070', '0007388', '0007514', '00085', '000850', '000892', '000975', '0009828', '00102', '001053', '00108', '0011054', '0011345', '00115', '001233', '00134', '001348', '00136', '001369', '0013881', '00139', '0014233', '0014867', '0014941', '001516', '00159', '00161', '00162', '00181', '0018119', '00186', '0019454', '0019796', '0020432', '00209', '002120', '00213', '002135', '002136', '00218', '0022277', '002281', '0022904', '00231', '0023203', '002385', '002434', '002442', '002470', '0024769', '002521', '002556', '0025751', '002584', '00265', '002784', '0027947', '002863', '002865', '0029270', '00298', '0030321', '0030681', '003072', '00317', '00323', '0032466', '0032538', '0033307', '0033660', '00339', '003507', '00351', '00353', '003531', '00356', '0036003', '003617', '0036520', '0037116', '003727', '003766', '0037737', '0038

In [ ]:
print(training_txt)

[[0, 0, 0, 0, 0, 3], [0, 0, 0, 0, 6, 9, 6], [0, 0, 0, 3, 3, 15], [0, 0, 0, 3, 6, 12], [0, 0, 0, 3, 21], [0, 0, 0, 6, 3], [0, 0, 0, 6, 15, 6], [0, 0, 0, 9, 0, 27, 12], [0, 0, 0, 9, 3, 18], [0, 0, 0, 9, 12, 12, 27], [0, 0, 0, 9, 27], [0, 0, 0, 12, 9, 24], [0, 0, 0, 15, 3], [0, 0, 0, 15, 9, 27, 21], [0, 0, 0, 15, 15], [0, 0, 0, 18, 3, 18], [0, 0, 0, 21, 0], [0, 0, 0, 21, 9, 24, 24], [0, 0, 0, 21, 15, 3, 12], [0, 0, 0, 24, 15], [0, 0, 0, 24, 15, 0], [0, 0, 0, 24, 27, 6], [0, 0, 0, 27, 21, 15], [0, 0, 0, 27, 24, 6, 24], [0, 0, 3, 0, 6], [0, 0, 3, 0, 15, 9], [0, 0, 3, 0, 24], [0, 0, 3, 3, 0, 15, 12], [0, 0, 3, 3, 9, 12, 15], [0, 0, 3, 3, 15], [0, 0, 3, 6, 9, 9], [0, 0, 3, 9, 12], [0, 0, 3, 9, 12, 24], [0, 0, 3, 9, 18], [0, 0, 3, 9, 18, 27], [0, 0, 3, 9, 24, 24, 3], [0, 0, 3, 9, 27], [0, 0, 3, 12, 6, 9, 9], [0, 0, 3, 12, 24, 18, 21], [0, 0, 3, 12, 27, 12, 3], [0, 0, 3, 15, 3, 18], [0, 0, 3, 15, 27], [0, 0, 3, 18, 3], [0, 0, 3, 18, 6], [0, 0, 3, 24, 3], [0, 0, 3, 24, 3, 3, 27], [0, 0, 3, 24, 1

In [ ]:
print('number of training images: ', len(training_img))
print('number of validation images: ', len(valid_img))

number of training images:  848
number of validation images:  152


In [ ]:
print(max_label_len)

7


In [ ]:
# pad each output label to maximum text length
 
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [ ]:
print(train_padded_txt)

[[ 0  0  0 ...  6  9  6]
 [ 0  0  0 ...  3 15 28]
 [ 0  0  0 ...  6 12 28]
 ...
 [ 0  9 12 ...  9 24 28]
 [ 0  9 12 ... 21  6 18]
 [ 0  9 12 ... 24 28 28]]


In [ ]:
print(len(train_padded_txt))

848


In [ ]:
# input with shape of height=64 and width=306
inputs = Input(shape=(64,306,1))
 

conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_6)
batch_norm_7 = BatchNormalization()(conv_7)
pool_7 = MaxPool2D(pool_size=(2, 1))(batch_norm_7)

conv_8 = Conv2D(512, (2,2), activation = 'relu')(pool_7)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
 

blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# this is the model we use for our prediction(after training)
prediction_model = Model(inputs, outputs)

In [ ]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

In [ ]:
# this is the model for training
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

In [ ]:
print(model.summary)

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7fa5a3f80510>>


In [ ]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

training_txt = np.array(training_txt)
valid_txt = np.array(valid_txt)

batch_size = 32
epochs = 14

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length],
          y=np.zeros(len(training_img)),
          batch_size=batch_size,
          epochs = epochs,
          validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]),
          verbose = 1)

14/14 [==============================] - 468s 32s/step - loss: 44.9593 - val_loss: 26.2752


In [ ]:
model.save_weights('/content/model_weights.hdf5')

In [ ]:
# load the model weights
model.load_weights('/content/model_weights (2).hdf5')

# predict outputs on validation images
prediction = prediction_model.predict(valid_img[:10])

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
for x in out:
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

ValueError: ignored